# Batch Scoring with OpenAI 
You can use a similar multithreading approach to make batch requests to the OpenAI API's completions endpoint. This can be particularly useful if you need to process a large number of requests in parallel to improve throughput and reduce waiting times.

In [ ]:
# install necessary packages
! pip install openai

In [2]:
from concurrent.futures import ThreadPoolExecutor, as_completed

# Decorator to add multithreading
def multithreaded(max_workers=5):
    def decorator(func):
        def wrapper(*args, **kwargs):
            with ThreadPoolExecutor(max_workers=max_workers) as executor:
                future_to_args = {executor.submit(func, arg): arg for arg in args[0]}
                results = []
                for future in as_completed(future_to_args):
                    arg = future_to_args[future]
                    try:
                        result = future.result()
                    except Exception as exc:
                        print(f'{arg} generated an exception: {exc}')
                    else:
                        results.append(result)
                return results
        return wrapper
    return decorator

In [3]:
import time
import os 
from openai import OpenAI
client = OpenAI()

# Set your OpenAI API key
client.api_key = os.environ["OPENAI_API_KEY"] #'your-api-key-here'

# Function to make a request to the OpenAI API
@multithreaded(max_workers=5)
def get_completion(prompt):
    try:
        response = client.completions.create(
        model="gpt-3.5-turbo-instruct",
        prompt=prompt,
        max_tokens=50,
        temperature=1.1
        )
        return response.choices[0].text.strip()
    except Exception as e:
        return f"Error: {e}"

# List of prompts to process
prompts = [
    "Tell me a joke.",
    "What's the weather like today?",
    "Explain the theory of relativity.",
    "What is the capital of France?",
    "Write a short story about a dragon.",
    "Translate 'Hello' to Spanish.",
    "What's the tallest mountain in the world?",
    "Who was Albert Einstein?",
    "Give me a recipe for chocolate cake.",
    "What are the benefits of meditation?"
]

# Using the decorated function to process prompts in parallel
start_time = time.time()
responses = get_completion(prompts)
end_time = time.time()

print("Responses:", responses)
print("Time taken:", end_time - start_time, "seconds")

Responses: ['Error: Connection error.', 'Error: Connection error.']
Time taken: 2.7098333835601807 seconds
